In [1]:
import os
%pwd

'/Users/andy/Developer/ecan_vsr/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/andy/Developer/ecan_vsr'

In [ ]:
from ecan.constants import *
from ecan.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __int__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories(self.config.artifacts_root)